In [75]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [101]:
import pandas as pd
import os
import sys
sys.path.append('../src/')
import vtn
from bokeh.io import output_notebook
import itertools 
output_notebook()

from ipywidgets import interact, interactive_output, interact_manual
import numpy as np
from scipy import optimize

Loading BokehJS ...

In [102]:
def load_data(n, datadir, filename):
    d_dict = {}
    for i in range(1,n+1):
        d_dict[f'{i}'] = pd.read_csv(os.path.join(datadir, filename.format(i=i)))
    return d_dict

# Initilize `VariableTimeNormalization` object

In [103]:
# Pair of experiments
exp_pairs = {
    'A':{
        'exps':['1','2'], 
        'type':'variable', 
        'colors':['blue', 'red']
    },
    'B':{
        'exps':['1','3'], 
        'type':'variable', 
        'colors':['blue', 'green']
    },
    'cat':{
        'exps':['1','4'], 
        'type':'constant', 
        'colors':['blue', 'orange']
    }
}

# Reaction orders with respect to each component
rxn_orders = {'A':1, 'B':1, 'cat':1}

# Initialize vtn object
exp = vtn.VariableTimeNormalization(
    df0=load_data(4, '../data/', 'exp_{i}.csv'), 
    exp_pairs=exp_pairs, 
    rxn_orders=rxn_orders, 
    product_name='P',
    
)

# Determine reaction orders
We can plot the the total variation in the absolute value of the discrete difference in the data over a range of reaction orders. The minimum of this plot indicates the order of the reaction.

There are two plotting functions available for total variation plotting:
- `plot_tv()`: plot total variable function `tv()` over specified range
- `plot_min_tv()`: find global minimum of `tv()` by brute force over specified grid

In [65]:
vtn.plot_tv(
    exp=exp,
    comps=['A','B','cat'],
    x0_arange=[0,2,0.25]
)

In [105]:
vtn.plot_min_tv(
    exp=exp,
    comps=['A','B','cat'],
    x0_arange=(0,2),
    n=25
)

# Generate VTN plots

All the data required to generate VTN plots are stored in `.df_vtn` dictionary. They keys of the dictionary correspond to experiment number. i.e. `.df_vtn['1']` is the DataFrame containing data for experiment `1`.

There are two plotting methods available to `VariableTimeNormalization` object.
- `plot_vtn()`: used for pairwise experiment plotting for determining order with respect to a component
- `plot_kobs()`: used for plotting product against normalized time from all exeriments to determine $k_{obs}$

In [108]:
exp = vtn.VariableTimeNormalization(
    load_data(4, '../data/', 'exp_{i}.csv'), 
    exp_pairs=exp_pairs, 
    rxn_orders={'A':.8, 'B':1.1, 'cat':.9}, 
    product_name='P'
)

exp.plot_vtn('A')
exp.plot_kobs()
exp.df_vtn['3']

,t,A,B,P,cat,t-1,[cat]^0.9,t[cat]^0.9,A-1,[A]^0.8,[A]^0.8∆t,∑[A]^0.8∆t,B-1,[B]^1.1,[B]^1.1∆t,∑[B]^1.1∆t,∆t,∑[A]^0.8[B]^1.1[cat]^0.9∆t
0,0.00,0.999,0.801,0.000,0.01,NaN,0.015849,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.33,0.915,0.715,0.086,0.01,0.00,0.015849,0.005230,0.999,0.840473,0.277356,0.277356,0.801,0.790204,0.260767,0.260767,0.33,0.003474
2,1.00,0.786,0.585,0.213,0.01,0.33,0.015849,0.015849,0.915,0.764775,0.512399,0.789755,0.715,0.667279,0.447077,0.707844,0.67,0.008893
3,1.92,0.663,0.464,0.337,0.01,1.00,0.015849,0.030430,0.786,0.672705,0.618889,1.408644,0.585,0.527015,0.484854,1.192698,0.92,0.014062
4,3.08,0.560,0.359,0.441,0.01,1.92,0.015849,0.048815,0.663,0.587369,0.681348,2.089992,0.464,0.403562,0.468131,1.660830,1.16,0.018420
5,4.38,0.482,0.284,0.515,0.01,3.08,0.015849,0.069418,0.560,0.516731,0.671750,2.761742,0.359,0.307611,0.399894,2.060724,1.30,0.021695
6,6.29,0.410,0.211,0.591,0.01,4.38,0.015849,0.099690,0.482,0.456312,0.871556,3.633298,0.284,0.230694,0.440625,2.501349,1.91,0.024881
7,11.54,0.310,0.108,0.691,0.01,6.29,0.015849,0.182897,0.410,0.384447,2.018345,5.651642,0.211,0.142279,0.746963,3.248312,5.25,0.029433
8,22.92,0.237,0.038,0.763,0.01,11.54,0.015849,0.363258,0.310,0.308574,3.511576,9.163219,0.108,0.060222,0.685331,3.933643,11.38,0.032784
9,41.67,0.208,0.009,0.792,0.01,22.92,0.015849,0.660425,0.237,0.261612,4.905229,14.068448,0.038,0.017309,0.324548,4.258191,18.75,0.034130


# OLS to determine $k_{obs}$

- `.kobs_line_format` can be specified when initializing the `vtn` object. This is the formula used for regression analysis to determine $k_{obs}$. This follows `R`-programming stype notation. By default this is set to `y ~ x-1`. This corresponds to fitting $y = mx$. The `-1` forces intercept to `0`.

- `.df_kobs_reg` is the DataFrame that contains data used for regression analysis to determine $k_{obs}$

- `.kobs_line` is a `statsmodel` object containing the OLS line. The output of OLS analysis can be generated using the `.summary()` method.

In [8]:
exp.kobs_line_format

'y ~ x-1'

In [9]:
exp.df_kobs_reg.head()

,P,∑[A]^1[B]^1[cat]^1∆t,y,x
1,0.150,0.004313,0.150,0.004313
2,0.273,0.007794,0.273,0.007794
3,0.388,0.011115,0.388,0.011115
4,0.477,0.013669,0.477,0.013669
5,0.534,0.015263,0.534,0.015263


In [10]:
exp.kobs_line.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 8.327e+05
Date:                Thu, 14 Feb 2019   Prob (F-statistic):           3.46e-78
Time:                        20:16:07   Log-Likelihood:                 156.22
No. Observations:                  36   AIC:                            -310.4
Df Residuals:                      35   BIC:                            -308.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x             34.8277      0.038    912.498      0.000      34.750      34.905
==============================================================================
Omnibus:                       22.711   Durbin-Watson:                   1.270
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.440
Skew:                          -1.573   Prob(JB):                     1.65e-09
Kurtosis:                       7.130   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""